In [3]:
import pandas as pd
import evaluate
from bert_score import score

In [16]:
df = pd.read_csv("../../evaluation_gams9b.csv")
df = df.dropna(subset=['expected_message', 'generated_message'])
generated_messages = df["generated_message"]
expected_messages = df["expected_message"]
df.head(3)

,user_message,expected_message,generated_message
0,B1: Opozorila Na hitri cesti Izola - Koper je ...,Na štajerski avtocesti proti Mariboru je zarad...,Na primorski avtocesti proti Ljubljani je zara...
1,B1: Zimske razmere Prelaz Vršič je zaprt. Na a...,Zaradi burje je na vipavski hitri cesti med ra...,Na vipavski hitri cesti je med Nanosom in Selo...
3,B1: Delo na cesti Dela na gorenjski avtocesti ...,Na primorski avtocesti proti Kopru je zaradi p...,Na območju rdeče stopnje je prepovedan promet ...


In [21]:
print("Evaluation by hand")
i = 1
print(expected_messages[i])
print("-"*200)
print(generated_messages[i])

Evaluation by hand
Zaradi burje je na vipavski hitri cesti med razcepom Nanos in priključkom Selo prepovedan promet za hladilnike in vozila s ponjavami, lažja od 8 ton. Zaprt je prelaz Vršič, na avstrijski strani prelaza Korensko sedlo pa so obvezne verige. Ponekod po državi je zaradi megle zmanjšana vidljivost.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Na vipavski hitri cesti je med Nanosom in Selom prepovedan promet za počitniške prikolice, hladilnike in vozila s ponjavami do 8 ton, burja 1. stopnje.


In [7]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=generated_messages, references=[[ref] for ref in expected_messages])
print(results)

{'bleu': 9.262142801994935e-15, 'precisions': [0.06636363636363636, 0.03577106518282989, 0.05379746835443038, 0.01948051948051948], 'brevity_penalty': 2.3321440255402565e-13, 'length_ratio': 0.03323714704416008, 'translation_length': 2200, 'reference_length': 66191}


In [22]:
from bert_score import score

generated_messages = [str(m) for m in generated_messages]
expected_messages = [str(m) for m in expected_messages]

assert len(generated_messages) == len(expected_messages)

P, R, F1 = score(
    cands=generated_messages,
    refs=expected_messages,
    model_type="xlm-roberta-base",  # model, ki podpira slovenščino
    lang="sl",                      # samo za združljivost
    rescale_with_baseline=False,   # ne uporabljaj baseline vrednosti
    idf=False,                      # brez IDF
    verbose=True,
)

print(f"BERTScore Precision (P): {P.mean().item():.4f}")
print(f"BERTScore Recall (R):    {R.mean().item():.4f}")
print(f"BERTScore F1:            {F1.mean().item():.4f}")



calculating scores...
computing bert embedding.


  0%|          | 0/15 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/15 [00:00<?, ?it/s]

done in 14.31 seconds, 65.83 sentences/sec
BERTScore Precision (P): 0.7758
BERTScore Recall (R):    0.6966
BERTScore F1:            0.7340


In [18]:
import evaluate

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=generated_messages, references=expected_messages)
print(results)


{'rouge1': np.float64(0.002932989152362467), 'rouge2': np.float64(0.0011701198500400603), 'rougeL': np.float64(0.002106692363024496), 'rougeLsum': np.float64(0.002076964016120725)}


ROUGE and BLEU rely on n-gram overlap (i.e., exact matching of words or sequences of words).T hese low values mean the generated messages share almost no exact words or phrases with the reference (expected) messages.

BLEU’s brevity penalty is extremely harsh: 2.33e-13, meaning the outputs are much shorter than the references (length_ratio ≈ 0.033), which kills the score.

This is especially problematic in free-form or abstractive generation, or when paraphrasing.

BERTScore does not require exact word overlap. It compares sentences based on semantic similarity using BERT embeddings. the results show that generated messages are semantically similar to the references. Even if the exact words or phrasing differ, the meaning is preserved.